# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [78]:
from random import choices, randint, random, choice, shuffle
from copy import deepcopy
from tqdm.notebook import trange
import numpy as np
import math

import lab9_lib

In [3]:
NUM_GENOMES = 1000
TOURNAMENT_SIZE = 2

In [4]:
class Individual:
    genomes: list
    fitness: float

    def __init__(self, genomes=None):
        if genomes == None:
            self.genomes = choices([0, 1], k=1000)
        else:
            self.genomes = genomes

    def mutate(self):
        mutated_genomes = deepcopy(self.genomes)
        index = choice(range(len(self.genomes)))
        if self.genomes[index] == 1:
            mutated_genomes[index] = 0
        else:
            mutated_genomes[index] = 1
        return Individual(genomes=mutated_genomes)


def select_parent(population) -> Individual:
    pool = choices(population, k=TOURNAMENT_SIZE)
    champion = max(pool, key=lambda i: i.fitness)
    return champion


def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    offspring_genotype = [ind1.genomes[i] if random() < 0.5 else ind2.genomes[i] for i in range(NUM_GENOMES)]
    return Individual(genomes=offspring_genotype)


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_GENOMES - 1)
    offspring = Individual(genomes=ind1.genomes[:cut_point] + ind2.genomes[cut_point:])
    return offspring

In [18]:
NUM_GENERATION = 30_000
PERCENTAGE_EXTINCTION = 0.75

POPULATION_SIZE = 50
OFFSPRING_SIZE = 25
ONLY_MUTATION_PROBABILITY = 0.8
STD_THRESHOLD = 0.0005

In [29]:
def ga_algorithm(problem_type, select_parent, xover, promoting_diversity=None):
    fitness = lab9_lib.make_problem(problem_type)
    population = [Individual() for _ in range(POPULATION_SIZE)]

    for i in population:
        i.fitness = fitness(i.genomes)

    population.sort(key=lambda i: i.fitness, reverse=True)

    best_individual = population[0]

    pbar = trange(0, NUM_GENERATION)
    for _ in pbar:
        pbar.set_description(f"Best-individual fitness: {best_individual.fitness}")

        if math.isclose(1, population[0].fitness):
            break

        offspring = list()

        # checking convergence
        population_fitness = [i.fitness for i in population]
        if promoting_diversity == 'extinction' and np.std(population_fitness) < STD_THRESHOLD:
            num_indivual_to_extinction = int(POPULATION_SIZE * PERCENTAGE_EXTINCTION)
            population = choices(population, k=POPULATION_SIZE - num_indivual_to_extinction)
            offspring = [Individual() for _ in range(num_indivual_to_extinction)]

        else:
            for _ in range(OFFSPRING_SIZE):
                if random() < ONLY_MUTATION_PROBABILITY:
                    # mutation
                    p = select_parent(population)
                    o = p.mutate()
                else:
                    # xover and mutation
                    p1 = select_parent(population)
                    p2 = select_parent(population)
                    o = xover(p1, p2).mutate()

                offspring.append(o)

        for i in offspring:
            i.fitness = fitness(i.genomes)

        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE]

        # save the new best individual
        if population[0].fitness > best_individual.fitness:
            best_individual = population[0]

    print(f'Solved with {fitness.calls:,} fitness calls')
    return best_individual

### Problem 1

Solution with GA, with Tournament Selection and uniform cross over

In [23]:
best_individual = ga_algorithm(1, select_parent, uniform_xover)

  0%|          | 0/30000 [00:00<?, ?it/s]

Solved with 27,850 fitness calls


Solution with GA, with Tournament Selection and one cut cross over

In [24]:
best_individual = ga_algorithm(1, select_parent, one_cut_xover)

  0%|          | 0/30000 [00:00<?, ?it/s]

Solved with 39,150 fitness calls


### Problem 2
#### Promoting Diversity with **Extinction**

To solve the problem 2, I added a promoting diversity with **extinction**

In [86]:
NUM_GENERATION = 30_000
PERCENTAGE_EXTINCTION = 0.85

POPULATION_SIZE = 50
OFFSPRING_SIZE = 25
ONLY_MUTATION_PROBABILITY = 0.6
STD_THRESHOLD = 0.0005

In [87]:
best_individual = ga_algorithm(2, select_parent, uniform_xover, promoting_diversity='extinction')

  0%|          | 0/30000 [00:00<?, ?it/s]

Solved with 237,446 fitness calls


In [88]:
best_individual = ga_algorithm(2, select_parent, one_cut_xover, promoting_diversity='extinction')

  0%|          | 0/30000 [00:00<?, ?it/s]

Solved with 360,544 fitness calls


### Problem 5
I added an implementation with Islands

In [82]:
NUM_GENERATION = 30_000
PERCENTAGE_EXTINCTION = 0.85

POPULATION_SIZE = 50
OFFSPRING_SIZE = 25
ONLY_MUTATION_PROBABILITY = 0.75
STD_THRESHOLD = 0.0005

In [83]:
best_individual = ga_algorithm(5, select_parent, uniform_xover, promoting_diversity='extinction')

  0%|          | 0/30000 [00:00<?, ?it/s]

Solved with 783,234 fitness calls


In [68]:
NUM_ISLANDS = 50
NUM_MIGRANTS = 25
MIGRATION_STEP = 5

NUM_GENERATION = 10_000
PERCENTAGE_EXTINCTION = 0.85

POPULATION_SIZE = 50
OFFSPRING_SIZE = 25
TOURNAMENT_SIZE = 2
ONLY_MUTATION_PROBABILITY = 0.75
STD_THRESHOLD = 0.0005

In [75]:
def ga_algorithm_island(problem_type, select_parent, xover, extinction=False):
    fitness = lab9_lib.make_problem(problem_type)

    populations = [[Individual() for _ in range(POPULATION_SIZE)] for _ in range(NUM_ISLANDS)]

    for population in populations:
        for i in population:
            i.fitness = fitness(i.genomes)

        population.sort(key=lambda i: i.fitness, reverse=True)

    best_individuals = [population[0] for population in populations]
    best_global_individual = max(best_individuals, key=lambda x: x.fitness)

    pbar = trange(0, NUM_GENERATION)
    for generation in pbar:
        # migration
        if (generation + 1) % MIGRATION_STEP == 0:
            # random islands
            shuffle(populations)
            for idx in range(0, NUM_ISLANDS - 1, 2):
                # swap
                tmp = populations[idx][:NUM_MIGRANTS]
                populations[idx + 1][:NUM_MIGRANTS] = populations[idx][:NUM_MIGRANTS]
                populations[idx + 1][:NUM_MIGRANTS] = tmp

        for i in range(NUM_ISLANDS):
            pbar.set_description(f"Best-individual across island fitness: {best_global_individual.fitness}")

            if math.isclose(1, populations[i][0].fitness):
                break

            offspring = list()

            # extinction
            population_fitness = [ind.fitness for ind in populations[i]]
            if extinction and np.std(population_fitness) < STD_THRESHOLD:
                num_indivual_to_extinction = int(POPULATION_SIZE * PERCENTAGE_EXTINCTION)
                populations[i] = choices(populations[i], k=POPULATION_SIZE - num_indivual_to_extinction)
                offspring = [Individual() for _ in range(num_indivual_to_extinction)]

            else:
                for _ in range(OFFSPRING_SIZE):
                    if random() < ONLY_MUTATION_PROBABILITY:
                        # mutation
                        p = select_parent(populations[i])
                        o = p.mutate()
                    else:
                        # xover and mutation
                        p1 = select_parent(populations[i])
                        p2 = select_parent(populations[i])
                        o = xover(p1, p2).mutate()

                    offspring.append(o)

            for ind in offspring:
                ind.fitness = fitness(ind.genomes)

            populations[i].extend(offspring)
            populations[i].sort(key=lambda ind: ind.fitness, reverse=True)
            populations[i] = populations[i][:POPULATION_SIZE]

            # save the new best individual
            if populations[i][0].fitness > best_individuals[i].fitness:
                best_individuals[i] = populations[i][0]

            best_global_individual = max(best_individuals, key=lambda ind: ind.fitness)

    print(f'Solved with {fitness.calls:,} fitness calls')
    return best_global_individual

In [72]:
best_individual = ga_algorithm_island(5, select_parent, uniform_xover, extinction=True)

  0%|          | 0/1000 [00:00<?, ?it/s]

Solved with 1,278,588 fitness calls


In [76]:
NUM_ISLANDS = 50
NUM_MIGRANTS = 25
MIGRATION_STEP = 50

NUM_GENERATION = 10_000
PERCENTAGE_EXTINCTION = 0.85

POPULATION_SIZE = 50
OFFSPRING_SIZE = 25
TOURNAMENT_SIZE = 2
ONLY_MUTATION_PROBABILITY = 0.75
STD_THRESHOLD = 0.0005

In [79]:
best_individual = ga_algorithm_island(5, select_parent, uniform_xover, extinction=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

Solved with 12,937,608 fitness calls


In [ ]:
NUM_ISLANDS = 50
NUM_MIGRANTS = 25
MIGRATION_STEP = 50

NUM_GENERATION = 10_000
PERCENTAGE_EXTINCTION = 0.85

POPULATION_SIZE = 50
OFFSPRING_SIZE = 25
TOURNAMENT_SIZE = 2
ONLY_MUTATION_PROBABILITY = 0.75
STD_THRESHOLD = 0.0005